In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# General imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers

In [32]:
# Defining a results visualization function
def visualize_training_results(history):
    '''
    From https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
    
    Input: keras history object (output from trained model)
    '''
    fig, (ax1, ax2) = plt.subplots(2, sharex=True)
    fig.suptitle('Model Results')

    # summarize history for accuracy
    ax1.plot(history.history['acc'])
    ax1.plot(history.history['val_acc'])
    ax1.set_ylabel('Accuracy')
    ax1.legend(['train', 'test'], loc='upper left')
    # summarize history for loss
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_ylabel('Loss')
    ax2.legend(['train', 'test'], loc='upper left')
    
    plt.xlabel('Epoch')
    plt.show()

Business Problem: 
Make a decision support tool for ER physicians looking at X_rays regarding lung issues for the first time

In [5]:
import PIL

In [6]:
image = PIL.Image.open('../input/chest-xray-pneumonia/chest_xray/test/NORMAL/IM-0001-0001.jpeg')

In [7]:
image

In [8]:
image.size

In [9]:
# Your code here; transform the image files and then load them into Keras as tensors 
# (be sure to perform a train-val-test split)
import keras

# Instantiating a generator object and normalizing the RGB values
traingen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
testgen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
valgen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

train_data = traingen.flow_from_directory(
    directory='../input/chest-xray-pneumonia/chest_xray/train',
    target_size=(150, 150),
    class_mode='categorical',
    batch_size=20,
    seed=42
)

test_data = testgen.flow_from_directory(
    directory='../input/chest-xray-pneumonia/chest_xray/test',
    target_size=(150, 150),
    class_mode='categorical',
    batch_size=20,
    seed=42
)

val_data = valgen.flow_from_directory(
    directory='../input/chest-xray-pneumonia/chest_xray/val',
    target_size=(150, 150),
    class_mode='categorical',
    batch_size=20,
    seed=42
)

**Baseline Model**

In [13]:

#baseline_model = keras.Sequential(name='dense')
#baseline_model.add(Dense(500, activation='relu', input_shape=(150,150, 3))
#baseline_model.add(Dense(250, activation='relu'))
#baseline_model.add(Dense(1, activation='sigmoid'))

In [18]:
input_shape = (150,150,3)
output_shape = 2

In [19]:
baseline = keras.Sequential(
    [
        keras.Input(shape=input_shape), # Don't always need this input separately
        layers.Flatten(), # need to flatten our images to be one long array
        layers.Dense(100, activation="relu"),
        layers.Dense(output_shape, activation="softmax"),
    ])

baseline.summary()

In [20]:
baseline.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['acc'])

In [22]:

baseline_results = baseline.fit_generator(train_data,
                              steps_per_epoch=100,
                              epochs=10,
                              validation_data=test_data,)


In [31]:
baseline_results.history

In [34]:
visualize_training_results(baseline_results)

**Adding another layer**

In [25]:
# Adding in the layers
two_hidden = keras.Sequential(
    [
        keras.Input(shape=input_shape), # Don't always need this input separately
        layers.Flatten(), # need to flatten our images to be one long array
        layers.Dense(100, activation='relu'),
        layers.Dense(50, activation='relu'),
        layers.Dense(output_shape, activation="softmax"),
    ])

two_hidden.summary()

In [26]:
# Compiling the two-layer model
two_hidden.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['acc'])

In [27]:
# Fit the two-layer model 
two_hidden_results = two_hidden.fit_generator(train_data,
                              steps_per_epoch=100,
                              epochs=10,
                              validation_data=test_data)